In [4]:
# Author: Vatsal Sanjay
# vatsalsanjay@gmail.com
# Physics of Fluids
# Last updated: Dec 24, 2024

import numpy as np
import os
import subprocess as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.ticker import StrMethodFormatter
import multiprocessing as mp
from functools import partial
import argparse  # Add at top with other imports
import sys
import matplotlib.colors as mcolors
np.set_printoptions(threshold=sys.maxsize)
GridsPerR = 128
lw = 2

This is used to reproduce simulation results

In [2]:
for t in range(8210,10000,10):
    data = np.load(f"/home/gridsan/yhe/postprocess/dataFiles_Order_1/{int(t):08d}_data.npz")
    name = f"/home/gridsan/yhe/postprocess/dataFiles_Order_1/{int(t):08d}.pdf"
    X =data['X']
    Y = data['Y']
    vel_rot   = data['vel_rot']
    omega_rot = data['omega_rot']
    f_rot     = data['f_rot']
    ux_rot    = data['ux_rot']
    uy_rot    = data['uy_rot']

    zmin=-1.0
    zmax=1.0
    rmin=-1.0
    rmax=1.0
    # ----------------------------------------------------------
    x_extent = [zmin, zmax]
    y_extent = [rmin, rmax]

    # For imshow, extent is [x_min, x_max, y_min, y_max].
    extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]

    dimension=np.shape(X)[0]
    surface=np.zeros((dimension,2))
    f_gradient=np.zeros(dimension)
    for i in range(0,dimension-1):
        for j in range(0,dimension):
            #Get the coordinates of points where the volume fraction gradient is highest.
            if abs(f_rot[i+1,j]-f_rot[i,j]) > f_gradient[j]:  
                f_gradient[j] = abs(f_rot[i+1,j]-f_rot[i,j])
                surface[j,0] = X[i,j]
                surface[j,1] = Y[i,j]

    AxesLabel, TickLabel = 50, 20
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(19.20, 10.80))

    # Draw the rotated interface in green, domain boundaries, etc.
    for ax in [ax1, ax2]:
        # Gray dashed lines now get swapped similarly if you want
        # them at z=0 or r=0.  If you simply want a bounding box,
        # note that x->z, y->r:
        ax.plot([-1.0, 1.0], [0, 0], '--', color='grey', linewidth=2)  # "horizontal" axis is z
        ax.plot([0, 0], [-1.0, 1.0], '-.', color='grey', linewidth=2)  # "vertical" axis is r

        # Domain box:
        ax.plot([-1.0, 1.0], [-1.0, -1.0], '-', color='black', linewidth=2)
        ax.plot([-1.0, 1.0], [1.0, 1.0], '-', color='black', linewidth=2)
        ax.plot([-1.0, -1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
        ax.plot([1.0, 1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
        surface_plot=[np.column_stack([surface[:,0], surface[:,1]]) for j in range(0,1)]
        line_segments = LineCollection(surface_plot, linewidths=4, colors='green')
        ax.add_collection(line_segments)

    # ----------------------------------------------------------
    # 5) Now show imshow with the rotated arrays and extents:
    # ----------------------------------------------------------
    cntrl1 = ax1.imshow(
        vel_rot, 
        cmap="Blues", 
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_vel,
        vmin=0.0,
        vmax=0.5
    )
    cntrl2 = ax2.imshow(
        omega_rot, 
        cmap="coolwarm", 
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=-1e0,
        vmax=1e0
    )

    # Equal aspect ensures squares in the new orientation
    for ax in [ax1, ax2]:
        ax.set_aspect('equal')
        ax.set_xlim(zmin, zmax)  # x range
        ax.set_ylim(rmin, rmax)  # y range

    # Titles and labels that match the new orientation
    ax1.set_title(fr'$t\sqrt{{g/\lambda}} = {t/1000.:.4f}$ (Velocity)', fontsize=TickLabel)
    # ax1.set_xlabel('$X$', fontsize=AxesLabel)
    # ax1.set_ylabel('$r$', fontsize=AxesLabel)

    ax2.set_title(r'Vorticity', fontsize=TickLabel)
    # ax2.set_xlabel('$z$', fontsize=AxesLabel)
    # ax2.set_ylabel('$r$', fontsize=AxesLabel)
    # Colorbars: place them below each subplot, for instance
    fig.subplots_adjust(bottom=0.2, wspace=0.3)  # more spacing for colorbars
    cbar_ax1 = fig.add_axes([0.125, 0.1, 0.35, 0.03])   # x,y,width,height in figure coords
    c1 = plt.colorbar(cntrl1, cax=cbar_ax1, orientation='horizontal')
    c1.ax.tick_params(labelsize=TickLabel)
    c1.set_label(r'$\|u/\sqrt{g\lambda}\|$', fontsize=AxesLabel)

    cbar_ax2 = fig.add_axes([0.57, 0.1, 0.35, 0.03])
    c2 = plt.colorbar(cntrl2, cax=cbar_ax2, orientation='horizontal')
    c2.ax.tick_params(labelsize=TickLabel)
    c2.set_label(r'$\omega\sqrt{\lambda/g}$', fontsize=AxesLabel)

    for ax in [ax1, ax2]:
        ax.axis('off')


    plt.savefig(name, bbox_inches="tight")
    plt.close()


KeyboardInterrupt: 

Plot analytic and simulation results separately

In [3]:
def u_analytic(x,y,t):   #Here u is u/sqrt(g\lambda)
    u_analytic=np.zeros(1)
    #u_analytic_x=np.zeros(1)
    #u_analytic_y=np.zeros(1)
    #u_analytic_x=(0.1/1.07*np.sqrt(np.pi/2)) * np.exp(2*np.pi*y/1.072266)*np.cos(2*np.pi*(x/1.072266)-(t-2.89)/1.18)
    #u_analytic_y=(0.1/1.07*np.sqrt(np.pi/2)) * np.exp(2*np.pi*y/1.072266)*np.sin(2*np.pi*(x/1.072266)-(t-2.89)/1.18)
    u_analytic=(0.1*np.pi) * np.exp(2*np.pi*y/1.07)*np.cos(2*np.pi*(x/1.07-t/1.18))
    return u_analytic

def comparison(t):
    data = np.load(f"/work/pi_vmathai_umass_edu/Yutian He/dataFiles_Order_1/{int(t):08d}_data.npz")
    t=t/1000.
    X =data['X']
    Y = data['Y']
    vel_rot   = data['vel_rot']
    omega_rot = data['omega_rot']
    f_rot     = data['f_rot']
    ux_rot    = data['ux_rot']
    uy_rot    = data['uy_rot']

    zmin=-1.0
    zmax=1.0
    rmin=-1.0
    rmax=1.0
    # ----------------------------------------------------------
    x_extent = [zmin, zmax]
    y_extent = [rmin, rmax]

    # For imshow, extent is [x_min, x_max, y_min, y_max].
    extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    
    dimension=np.shape(X)[0]
    surface=np.zeros((dimension,2))
    percentage=np.zeros((dimension,dimension))
    f_gradient=np.zeros(dimension)
    u_th_field=np.zeros((dimension,dimension))
    for i in range(0,dimension-1):
        for j in range(0,dimension):
            #Get the coordinates of points where the volume fraction gradient is highest.
            if abs(f_rot[i+1,j]-f_rot[i,j]) > f_gradient[j]:  
                f_gradient[j] = abs(f_rot[i+1,j]-f_rot[i,j])
                surface[j,0] = X[i,j]
                surface[j,1] = Y[i,j]


    for i in range(dimension):
        for j in range(dimension):
            x=X[i,j]
            y=Y[i,j]
            u=vel_rot[i,j]
            u_th=abs(u_analytic(x,y,t))
            u_th_field[i,j]=u_th
            percentage[i,j]=abs((u-u_th)/u_th)
            if y > surface[j,1]:
                percentage[i,j]=0
                u_th_field[i,j]=0
   
    return [percentage,surface,X,dimension,u_th_field,vel_rot]

def surface_analytic(x,t):
    eta=0
    eta=0.05*np.cos(np.pi/2 * x/0.25) * np.cos(2*np.pi/1.18 *t)
    return eta

for t in range(0,10000,10):
    for plot_range in np.array((0.2,0.5))
        name = f"/home/yutianhe_umass_edu/basilisk/src/codes/Standing Waves/comparison/png/{int(t):08d}.png"
        [percentage,surface,X,dimension,u_th_field,vel_rot] = comparison(t)
        AxesLabel, TickLabel = 50, 20

        zmin=-1.0
        zmax=1.0
        rmin=-1.0
        rmax=1.0
        # ----------------------------------------------------------
        x_extent = [zmin, zmax]
        y_extent = [rmin, rmax]

        # For imshow, extent is [x_min, x_max, y_min, y_max].
        extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
        extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(19.20, 10.80))

        # Draw the rotated interface in green, domain boundaries, etc.
        for ax in [ax1, ax2]:
            # Gray dashed lines now get swapped similarly if you want
            # them at z=0 or r=0.  If you simply want a bounding box,
            # note that x->z, y->r:
            ax.plot([-1.0, 1.0], [0, 0], '--', color='grey', linewidth=2)  # "horizontal" axis is z
            ax.plot([0, 0], [-1.0, 1.0], '-.', color='grey', linewidth=2)  # "vertical" axis is r

            # Domain box:
            ax.plot([-1.0, 1.0], [-1.0, -1.0], '-', color='black', linewidth=2)
            ax.plot([-1.0, 1.0], [1.0, 1.0], '-', color='black', linewidth=2)
            ax.plot([-1.0, -1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
            ax.plot([1.0, 1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
            surface_plot = [np.column_stack([surface[:,0], surface[:,1]]) for j in range(0,1)]
            surface_analytic_plot = [np.column_stack([X[512,:], surface_analytic(X[512,:],t/1000.)]) for j in range(0,1)]
            if ax == ax1:

                line_segments = LineCollection(surface_plot, linewidths=4, colors='green')
                ax.add_collection(line_segments)
            else:
                line_segments_analytic = LineCollection(surface_analytic_plot, linewidths=4, colors='red')
                ax.add_collection(line_segments_analytic)

        # ----------------------------------------------------------
        # 5) Now show imshow with the rotated arrays and extents:
        # ----------------------------------------------------------
        cntrl1 = ax1.imshow(
            vel_rot, 
            cmap="Blues", 
            interpolation='bilinear', 
            origin='lower', 
            extent=extent_vel,
            vmin=0.0,
            vmax=plot_range
        )

        cntrl2 = ax2.imshow(
        u_th_field, 
        cmap="Blues", 
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=0.0,
        vmax=plot_range
        )


        # Equal aspect ensures squares in the new orientation
        for ax in [ax1, ax2]:
            ax.set_aspect('equal')
            ax.set_xlim(zmin, zmax)  # x range
            ax.set_ylim(rmin, rmax)  # y range

        # Titles and labels that match the new orientation
        ax1.set_title(fr'$t\sqrt{{g/\lambda}} = {t/1000:.4f}$ (Velocity)', fontsize=TickLabel)
        # ax1.set_xlabel('$X$', fontsize=AxesLabel)
        # ax1.set_ylabel('$r$', fontsize=AxesLabel)

        ax2.set_title(r'analytic', fontsize=TickLabel)
        # ax2.set_xlabel('$z$', fontsize=AxesLabel)
        # ax2.set_ylabel('$r$', fontsize=AxesLabel)
        # Colorbars: place them below each subplot, for instance

        fig.subplots_adjust(bottom=0.2, wspace=0.3)  # more spacing for colorbars
        cbar_ax1 = fig.add_axes([0.125, 0.1, 0.35, 0.03])   # x,y,width,height in figure coords
        c1 = plt.colorbar(cntrl1, cax=cbar_ax1, orientation='horizontal')
        c1.ax.tick_params(labelsize=TickLabel)
        c1.set_label(r'$\|u/\sqrt{g\lambda}\|$', fontsize=AxesLabel)

        cbar_ax2 = fig.add_axes([0.57, 0.1, 0.35, 0.03])
        c2 = plt.colorbar(cntrl2, cax=cbar_ax2, orientation='horizontal')
        c2.ax.tick_params(labelsize=TickLabel)
        c2.set_label(r'$\|u/\sqrt{g\lambda}\|$', fontsize=AxesLabel)

        for ax in [ax1, ax2]:
            ax.axis('off')


        plt.savefig(name, bbox_inches="tight")
        plt.close()



Plot them in one figure

In [ ]:
def u_analytic(x,y,t):   #Here u is u/sqrt(g\lambda)
    u_analytic=np.zeros(1)
    u_analytic=(0.2/1.07) * np.exp(2*np.pi*y/1.072266)*np.cos(2*np.pi*(x/1.072266)-(t-2.89)/1.18)
    return u_analytic

def comparison(t):
    data = np.load(f"/home/gridsan/yhe/postprocess/dataFiles_Order_1/{int(t):08d}_data.npz")
    t=t/1000.
    X =data['X']
    Y = data['Y']
    vel_rot   = data['vel_rot']
    omega_rot = data['omega_rot']
    f_rot     = data['f_rot']
    ux_rot    = data['ux_rot']
    uy_rot    = data['uy_rot']

    zmin=-1.0
    zmax=1.0
    rmin=-1.0
    rmax=1.0
    # ----------------------------------------------------------
    x_extent = [zmin, zmax]
    y_extent = [rmin, rmax]

    # For imshow, extent is [x_min, x_max, y_min, y_max].
    extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    
    dimension=np.shape(X)[0]
    surface=np.zeros((dimension,2))
    percentage=np.zeros((dimension,dimension))
    f_gradient=np.zeros(dimension)
    u_th_field=np.zeros((dimension,dimension))
    for i in range(0,dimension-1):
        for j in range(0,dimension):
            #Get the coordinates of points where the volume fraction gradient is highest.
            if abs(f_rot[i+1,j]-f_rot[i,j]) > f_gradient[j]:  
                f_gradient[j] = abs(f_rot[i+1,j]-f_rot[i,j])
                surface[j,0] = X[i,j]
                surface[j,1] = Y[i,j]


    for i in range(dimension):
        for j in range(dimension):
            x=X[i,j]
            y=Y[i,j]
            u=vel_rot[i,j]
            u_th=abs(u_analytic(x,y,t))
            u_th_field[i,j]=u_th
            percentage[i,j]=abs((u-u_th)/u_th)
            if y > surface[j,1]:
                percentage[i,j]=0
                u_th_field[i,j]=0
   
    return [percentage,surface,X,dimension,u_th_field,vel_rot]

def surface_analytic(x,t):
    eta=0
    eta=0.05*np.cos(np.pi/2 * x/0.25) * np.cos(2*np.pi/1.18 *t)
    return eta

for t in range(8210,10000,10):
    name = f"/home/gridsan/yhe/postprocess/dataFiles_Order_1/comparison/{int(t):08d}.pdf"
    [percentage,surface,X,dimension,u_th_field,vel_rot] = comparison(t)
    AxesLabel, TickLabel = 50, 20

    zmin=-1.0
    zmax=1.0
    rmin=-1.0
    rmax=1.0
    # ----------------------------------------------------------
    x_extent = [zmin, zmax]
    y_extent = [rmin, rmax]

    # For imshow, extent is [x_min, x_max, y_min, y_max].
    extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(19.20, 10.80))

    # Draw the rotated interface in green, domain boundaries, etc.
    for ax in [ax1, ax2]:
        # Gray dashed lines now get swapped similarly if you want
        # them at z=0 or r=0.  If you simply want a bounding box,
        # note that x->z, y->r:
        ax.plot([-1.0, 1.0], [0, 0], '--', color='grey', linewidth=2)  # "horizontal" axis is z
        ax.plot([0, 0], [-1.0, 1.0], '-.', color='grey', linewidth=2)  # "vertical" axis is r

        # Domain box:
        ax.plot([-1.0, 1.0], [-1.0, -1.0], '-', color='black', linewidth=2)
        ax.plot([-1.0, 1.0], [1.0, 1.0], '-', color='black', linewidth=2)
        ax.plot([-1.0, -1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
        ax.plot([1.0, 1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
        surface_plot = [np.column_stack([surface[:,0], surface[:,1]]) for j in range(0,1)]
        surface_analytic_plot = [np.column_stack([X[512,:], surface_analytic(X[512,:],t/1000.)]) for j in range(0,1)]
        if ax == ax1:
            
            line_segments = LineCollection(surface_plot, linewidths=4, colors='green')
            ax.add_collection(line_segments)
        else:
            line_segments_analytic = LineCollection(surface_analytic_plot, linewidths=4, colors='red')
            ax.add_collection(line_segments_analytic)

    # ----------------------------------------------------------
    # 5) Now show imshow with the rotated arrays and extents:
    # ----------------------------------------------------------
    cntrl1 = ax1.imshow(
        vel_rot, 
        cmap="Blues", 
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_vel,
        vmin=0.0,
        vmax=0.5
    )
    
    cntrl2 = ax2.imshow(
    u_th_field, 
    cmap="Blues", 
    interpolation='bilinear', 
    origin='lower', 
    extent=extent_omega,
    vmin=0.0,
    vmax=0.5
    )


    # Equal aspect ensures squares in the new orientation
    for ax in [ax1, ax2]:
        ax.set_aspect('equal')
        ax.set_xlim(zmin, zmax)  # x range
        ax.set_ylim(rmin, rmax)  # y range

    # Titles and labels that match the new orientation
    ax1.set_title(fr'$t\sqrt{{g/\lambda}} = {t/1000:.4f}$ (Velocity)', fontsize=TickLabel)
    # ax1.set_xlabel('$X$', fontsize=AxesLabel)
    # ax1.set_ylabel('$r$', fontsize=AxesLabel)
    
    ax2.set_title(r'analytic', fontsize=TickLabel)
    # ax2.set_xlabel('$z$', fontsize=AxesLabel)
    # ax2.set_ylabel('$r$', fontsize=AxesLabel)
    # Colorbars: place them below each subplot, for instance
    
    fig.subplots_adjust(bottom=0.2, wspace=0.3)  # more spacing for colorbars
    cbar_ax1 = fig.add_axes([0.125, 0.1, 0.35, 0.03])   # x,y,width,height in figure coords
    c1 = plt.colorbar(cntrl1, cax=cbar_ax1, orientation='horizontal')
    c1.ax.tick_params(labelsize=TickLabel)
    c1.set_label(r'$\|u/\sqrt{g\lambda}\|$', fontsize=AxesLabel)
    
    cbar_ax2 = fig.add_axes([0.57, 0.1, 0.35, 0.03])
    c2 = plt.colorbar(cntrl2, cax=cbar_ax2, orientation='horizontal')
    c2.ax.tick_params(labelsize=TickLabel)
    c2.set_label(r'$\|u/\sqrt{g\lambda}\|$', fontsize=AxesLabel)

    for ax in [ax1, ax2]:
        ax.axis('off')


    plt.savefig(name, bbox_inches="tight")
    plt.close()

